# Qualtrics integration and pipeline
Demonstration of how we will generate archetypes from the *Qualtrics* responses both for the current car as well as for each vintage (i.e. archetype defined by `vehicle_type` and `fuel_type`)

In [ ]:
##### implement logging #####
"https://docs.python.org/3/howto/logging.html"
import logging

logging.basicConfig(format='%(asctime)s %(levelname)s %(funcName)s %(message)s', filename='logging.log', level=logging.INFO)

## Qualtrics

In [ ]:
import os

from QualtricsAPI.Setup import Credentials

from QualtricsAPI import Responses


In [ ]:
TOKEN = os.environ.get("QUALTRICS_API_KEY")
DATA_CENTER = os.environ.get("QUALTRICS_BASE_URL").split(".")[0]


In [ ]:
Credentials().qualtrics_api_credentials(token=TOKEN, data_center=DATA_CENTER)


In [ ]:
r = Responses()
survey_id = "SV_6u4YzY24OAfhWfQ"
df_pre_study = r.get_survey_responses("SV_6u4YzY24OAfhWfQ")


In [ ]:
df_pre_study.head()


## Workflow

In [ ]:
import tcsscraper.scrape as tcs

from tcsscraper.helper import Car


In [ ]:
Car.vehicle_classes


In [ ]:
Car.fuel_types


In [ ]:
car = Car("Luxusklasse", "Benzin", 10)
print(car)


In [ ]:
similar_cars = tcs.get_cars(
    car, km=20e3, canton="ZH", similar={"flag": True, "buffer": 1}, verbose=True
)


In [ ]:
len(similar_cars)


In [ ]:
def remove_unit(string):
    """Removes unit from string like 17'000 CHF/year -> float(17e3)

    Args:
        string (str): string with unit
    """
    number, unit = string.split(" ")
    number = number.replace("'", "")
    return float(number)


In [ ]:
import pandas as pd


def generate_archetype(similar_cars, ndigits=2):
    """Generates an archetypical car from a list of similar cars

    Args:
        similar_cars (list of dict): returned by get_cars()
        ndigits (int): round average values to
    """
    car_attributes = []
    for c in similar_cars:
        costs = c["costs"]
        attrs = {
            "fix_cost": remove_unit(costs["Fixe Kosten"]),
            "variable_cost": remove_unit(costs["Variable Kosten"]),
            "cost_per_km": remove_unit(costs["Kilometerkosten"]),
        }
        car_attributes.append(attrs)

    car_attributes = pd.DataFrame(car_attributes)

    archetype = dict(round(car_attributes.mean(), ndigits=ndigits))

    return archetype


In [ ]:
archetype = generate_archetype(similar_cars)
archetype


## Pipeline

### User specific archetype

In [ ]:
## generate data.frame with variables to feed to get_cars
df_qualt = pd.DataFrame(
    {
        "vehicle_type": ["SUV S", "Luxusklasse", "Mittelklasse"],
        "fuel_type": ["Elektro", "Benzin", "Benzin"],
        "fuel_consumption": [14, 7, 2],  ## if electric => kWh/100km
        "km": [15e3, 20e3, 22e3],
        "canton": ["VD", "AI", "ZG"],
    }
)

df_qualt.head()


In [ ]:
def helper(row, max_buffer=5, verbose=True, headless=True):
    car = Car(row["vehicle_type"], row["fuel_type"], row["fuel_consumption"])
    init = 1
    if verbose:
        print("---\n{}\n---".format(car))
    while init < max_buffer + 1:
        try:
            sc = tcs.get_cars(
                car,
                row["km"],
                row["canton"],
                similar={"flag": True, "buffer": init},
                verbose=verbose,
                headless=headless
            )
        except Exception:
            init += 1
            if verbose:
                print("increaseing buffer to {}".format(init))
            continue
        return sc
    return None


In [ ]:
df_qualt["similar_cars"] = df_qualt.apply(lambda row: helper(row), axis=1)


In [ ]:
## generate archetypes
df_qualt["archetype"] = df_qualt.apply(
    lambda row: generate_archetype(row["similar_cars"]), axis=1
)
df_qualt["archetype"]


In [ ]:
## unpack stuff
df_qualt = df_qualt.assign(
    archetype_fix_cost=lambda x: [y["fix_cost"] for y in x["archetype"]],
    archetype_variable_cost=lambda x: [y["variable_cost"] for y in x["archetype"]],
    archetype_cost_per_km=lambda x: [y["cost_per_km"] for y in x["archetype"]],
)


### Generic archetypes

In [33]:
import numpy as np
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from tcsscraper.helper import NoSimilarCar
import pickle

## pack everything into a function
def generate_generic_archetypes(km=15e3, canton="ZH", verbose=True):
    """Generate archetype for each vehicle_class x fuel_type combination

    Args:
        km (int, optional): archetypes' annual mileage (reference). Defaults to 15e3.
        canton (str, optional): archetypes' domicile (reference). Defaults to 'AG'.
        verbose (bool, optional): neo. Defaults to True.

    Returns:
        pandas.DataFrame: containing vehicle_class, fuel_type, car_objects, as well as the archetypes (as returned by generate archetype)
    """
    ## generate df of all possible combinations
    archs = np.stack(np.meshgrid(Car.vehicle_classes, Car.fuel_types), axis=-1).reshape(
        -1, 2
    )
    df_archs = pd.DataFrame(archs)
    df_archs.columns = ["vehicle_class", "fuel_type"]

    df_archs["car_objects"] = df_archs.apply(
        lambda row: Car(row["vehicle_class"], row["fuel_type"], fuel_consumption=None),
        axis=1,
    )

    cars = []
    for c in df_archs["car_objects"]:
        if verbose:
            print("---\n{}\n---".format(c))
        try:
            car = tcs.get_cars(
                car_object=c,
                canton=canton,
                km=km,
                similar={"flag": False},
                headless=True,
                verbose=verbose,
            )
        except (NoSimilarCar, NoSuchElementException, StaleElementReferenceException) as e:
            logging.warning("Exception for {} x {}".format(c.vehicle_class, c.fuel_type))
            car = None
        cars.append(car)

    ## save
    with open("../../data/cars", "wb") as fp:
        pickle.dump(cars, fp)

    # ## load
    # with open("../../data/cars", "rb") as fp:
    #     cars = pickle.load(fp)

    generic_archetypes = []
    for c in cars:  ## is list of similar cars
        if c is None:
            generic_archetypes.append(None)
            continue
        ga = generate_archetype(c)
        generic_archetypes.append(ga)

    df_archs["generic_archetypes"] = generic_archetypes

    return df_archs


In [ ]:
# ## copy from above... to run after cars has been dumped
# archs = np.stack(np.meshgrid(Car.vehicle_classes, Car.fuel_types), axis=-1).reshape(
#     -1, 2
# )
# df_archs = pd.DataFrame(archs)
# df_archs.columns = ["vehicle_class", "fuel_type"]

# df_archs["car_objects"] = df_archs.apply(
#     lambda row: Car(row["vehicle_class"], row["fuel_type"], fuel_consumption=None),
#     axis=1,
# )

# ## load
# with open("../../data/cars", "rb") as fp:
#     cars = pickle.load(fp)

# generic_archetypes = []
# for c in cars:  ## is list of similar cars
#     if c is None:
#         generic_archetypes.append(None)
#         continue
#     ga = generate_archetype(c)
#     generic_archetypes.append(ga)

# df_archs["generic_archetypes"] = generic_archetypes

# with open("../../data/df_archs", "wb") as fp:
#     pickle.dump(df_archs)

In [34]:
generic_archs = generate_generic_archetypes()

---
vehicle_type:     Mikroklasse
fuel_type:        Benzin
fuel_consumption: None
---
Extracted Toyota Aygo 1.0 VVT-i x-play man.
Extracted Toyota Aygo 1.0 VVT-i x-play x-shift
Extracted Renault Twingo Sce 65 Zen
Extracted Hyundai i10 1.0 T-Gdi N-Line
Extracted VW Up 1.0 MPI move up!
Extracted Kia Picanto 1.0 CVVT Picanto
Extracted Kia Picanto 1.0 T-Gdi GT-Line
Extracted Kia Picanto 1.0 CVVT First Edition Auto.
Extracted Hyundai i10 1.0 Pica
Extracted Kia Picanto 1.2 CVVT Power
Extracted VW Up 1.0 TSI GTI
Extracted Kia Picanto 1.2 CVVT Power Aut.
Extracted Hyundai i10 1.2 Amplia
Extracted Hyundai i10 1.0 Origo auto.
Extracted Hyundai i10 1.2 Amplia Auto.
Extracted FIAT Panda 0.9 TwinAir 4x4 Wild
Extracted FIAT 500 Abarth 695
Extracted FIAT 500 Abarth 695 MTA
---
vehicle_type:     Kleinwagen
fuel_type:        Benzin
fuel_consumption: None
---
Extracted Mitsubishi Space Star 1.2 Pure
Extracted Toyota Yaris 1.5 Comfort
Extracted Toyota Yaris 1.5 Comfort Multidrive S
Extracted Mitsubishi S

NoSuchElementException: Message: Unable to locate element: select


## Remarks
- `fuel_consumption` with `fuel_type='electric'` see **Normverbrauch**: https://www.verbrauchskatalog.ch/index.php => higher values and unit kWh/100km